Limpieza de datos Rooms

In [557]:
import pandas as pd
import numpy as np
import re 

In [715]:
data_location = "./Data/properati.csv"
data = pd.read_csv(data_location, sep=",")
print(data.dtypes)
data.head(3)

Unnamed: 0                      int64
operation                      object
property_type                  object
place_name                     object
place_with_parent_names        object
country_name                   object
state_name                     object
geonames_id                   float64
lat-lon                        object
lat                           float64
lon                           float64
price                         float64
currency                       object
price_aprox_local_currency    float64
price_aprox_usd               float64
surface_total_in_m2           float64
surface_covered_in_m2         float64
price_usd_per_m2              float64
price_per_m2                  float64
floor                         float64
rooms                         float64
expenses                      float64
properati_url                  object
description                    object
title                          object
image_thumbnail                object
dtype: objec

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...


In [716]:
data["rooms_completo"] =data["rooms"]
data_room_null_description = data.description.astype(str).str.lower()
display(data_room_null_description)

0         2 ambientes tipo casa planta baja por pasillo,...
1         venta de departamento en décimo piso al frente...
2         2 ambientes 3er piso lateral living comedor am...
3         ph 3 ambientes con patio. hay 3 deptos en lote...
4         departamento con fantástica iluminación natura...
                                ...                        
121215    torre forum alcorta - máxima categoría.impecab...
121216    excelente e impecable casa en venta en las lom...
121217    venta departamento ambiente divisible a estren...
121218    2 amb al contrafrente,  luminoso. el departame...
121219    departamento de 77 m2 apto profesional. edific...
Name: description, Length: 121220, dtype: object

In [717]:
print("Primer grupo de filas sin rooms")
data["rooms_completo"].isnull().value_counts()

Primer grupo de filas sin rooms


True     73830
False    47390
Name: rooms_completo, dtype: int64

Seteamos los que son monoambientes
Primero buscamos las palabras que identifiquen al monoambiente. 
Luego buscamos las coincidencias dentro del universo de coincidencias del campo descripción
donde hubo coincidencias, le asignamos 1. 

In [718]:
def fun_buscar_patt(data_validar,patt_room):
    patt_regex = re.compile(patt_room)
    data_match = data_validar.apply(lambda x:  x if x is np.NaN else patt_regex.search(x))
    return data_match

In [719]:
def fun_buscar_disminuir(data_room_null_description, patt,restar):
    ###se pasa la palabra y se asigna###
    data_encontrada= fun_buscar_patt(data_room_null_description,patt)
    encontradas = data_encontrada[data_encontrada.notnull()].apply(lambda x: x.group(1))
    room_null_mask = data["rooms_completo"].isnull()
    data.loc[room_null_mask,"rooms_completo"]= encontradas[room_null_mask].apply(lambda x: x if x is np.NaN else (float(x)-restar))
    print("Disminución en función a los ambientes tipo:",patt)
    result = data["rooms_completo"].isnull().value_counts()
    return result

In [720]:
def fun_buscar_disminuir_mapper(data_room_null_description, patt,restar):
    ###se pasa la palabra y se asigna y se mapea el texto###
    data_encontrada= fun_buscar_patt(data_room_null_description,patt)
    encontradas = data_encontrada[data_encontrada.notnull()].apply(lambda x: x.group(1))
    room_null_mask = data["rooms_completo"].isnull()
    numcol_cat_mapper = {
       'un': 1,'uun':1, 'uno': 1,'dos': 2,'tres': 3,'cuatro':4,
        'cinco':5,'sinco':5,'seis':6,'ceis':6,'siete':7,
        'ciete':7,'ocho':8,'nueve':9,'diez':10,'once':11,
        'doce':12,'trece':13,'siete':14,'quince':15,
        'dieciseis':16,'diecisiete':17,'dieciocho':18,'diecinueve':19,
        'veinte':20
    }
    encontradas1 = encontradas.map(numcol_cat_mapper)
    data.loc[room_null_mask,"rooms_completo"]= encontradas1[room_null_mask].apply(lambda x: x if x is np.NaN else (float(x)-restar))
    print("Disminución en función a los ambientes :",patt)
    result = data["rooms_completo"].isnull().value_counts()
    return result

In [721]:
data_room_match = fun_buscar_patt(data_room_null_description,r"(mono ambiente|monoambiente|mono-ambiente)")
data_room_match_mask = data_room_match.notnull()
data_room_match[data_room_match_mask]=0
data_room_mask = data["rooms_completo"].isnull()

data.loc[data_room_match_mask,"rooms_completo"] = data_room_match[data_room_match_mask].apply(lambda x: x if x is np.NaN else x)
print("Primera disminución Monoambientes")
display(data["rooms_completo"].isnull().value_counts())

Primera disminución Monoambientes


True     69080
False    52140
Name: rooms_completo, dtype: int64

In [722]:
resultado = fun_buscar_disminuir(data_room_null_description,r"([1-9]{1,3}) *(habitaci)",0)
print(resultado)

Disminución en función a los ambientes tipo: ([1-9]{1,3}) *(habitaci)
True     67503
False    53717
Name: rooms_completo, dtype: int64


In [723]:
resultado = fun_buscar_disminuir(data_room_null_description,r"([1-9]{1,3}) *(dormitorio|domitorio)",0)
print(resultado)

Disminución en función a los ambientes tipo: ([1-9]{1,3}) *(dormitorio|domitorio)
False    77068
True     44152
Name: rooms_completo, dtype: int64


In [724]:
resultado = fun_buscar_disminuir_mapper(data_room_null_description,r"(\w+) *(dormitorio|domitorio)",0)
print(resultado)

Disminución en función a los ambientes : (\w+) *(dormitorio|domitorio)
False    88578
True     32642
Name: rooms_completo, dtype: int64


In [725]:
resultado = fun_buscar_disminuir(data_room_null_description,r"([1-9]{1,3}) *(ambiente)",1)
print(resultado)

Disminución en función a los ambientes tipo: ([1-9]{1,3}) *(ambiente)
False    98428
True     22792
Name: rooms_completo, dtype: int64


In [726]:
resultado = fun_buscar_disminuir_mapper(data_room_null_description,r"(\w+) *(ambiente)",1)
print(resultado)

Disminución en función a los ambientes : (\w+) *(ambiente)
False    100081
True      21139
Name: rooms_completo, dtype: int64


In [727]:
resultado = fun_buscar_disminuir(data_room_null_description,r"([1-9]{1}) *(amb)",1)
print(resultado)

Disminución en función a los ambientes tipo: ([1-9]{1}) *(amb)
False    101739
True      19481
Name: rooms_completo, dtype: int64


In [728]:
resultado = fun_buscar_disminuir(data_room_null_description,r"([1-9]{1}) *(dorm)",0)
print(resultado)


Disminución en función a los ambientes tipo: ([1-9]{1}) *(dorm)
False    102378
True      18842
Name: rooms_completo, dtype: int64


In [729]:
resultado = fun_buscar_disminuir(data_room_null_description,r"([1-9]{1}) *(suite)",0)
print(resultado)

Disminución en función a los ambientes tipo: ([1-9]{1}) *(suite)
False    102488
True      18732
Name: rooms_completo, dtype: int64


In [743]:
data_room_search = data["description"][data["rooms_completo"].isnull()]
data_room_search.sample(8)

106097    GARCIA ANDREU -Negocios Inmobiliarios- ofrece ...
97032     Impecable casa totalmente reciclada a nuevo. P...
20198     Corredor Responsable: Mariano Aufseher - CUCIC...
76489     *INMOBILIARIA INTEGRANTE DEL CIC* Local frente...
21389     COMPLEJO DE PH SIN EXPENSAS A ESTRENAR CERCA D...
46709     ¿Querés vivir rodeado de paz ,paisajes natural...
105106             Casa 100m² con Pavimento, Tucumán, Lules
80910     NUEVAS RAICES vende edificio MAITEN ubicado en...
Name: description, dtype: object

In [755]:
#76874,100557,35002 = 2
#46551 = 1
#48411 = 4
#66284,75074 = cant, palabra dormitorio
idreg= 70269
print(data.title[idreg])
display(data.loc[idreg:idreg,["property_type","rooms_completo","description"]])
display(data_room_null_description[idreg])
data_finded= fun_buscar_patt(data["description"][data["rooms_completo"].isnull()],"r(dormitorio)")
print("--___----")
display(data_finded[data_finded.notnull()])

PH al frente


,property_type,rooms_completo,description
70269,PH,NaN,"PH al frente , 6 x 19 m , sup cub 85 m2 loza ..."


' ph al frente , 6 x 19 m , sup cub 85 m2 loza , espacio aereo libre.4 entradas , por el porch, por el garaje, por el pasillo comun con el ph del fondo, al garaje y al comedordormitorio , cocina , comedor con pisos de mosaico , baño6x3 garaje, 6x 5 patio , '

--___----


70269    <re.Match object; span=(173, 184), match='rdor...
Name: description, dtype: object

In [747]:
data.loc[idreg:idreg+1,:]

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail,rooms_completo
97032,97032,sell,house,Florida,|Argentina|Bs.As. G.B.A. Zona Norte|Vicente Ló...,Argentina,Bs.As. G.B.A. Zona Norte,3433908.0,NaN,NaN,...,NaN,1970.588235,NaN,NaN,NaN,http://www.properati.com.ar/1bjhs_venta_casa_f...,Impecable casa totalmente reciclada a nuevo. P...,Estanislao del Campo al 1600 - Florida Belgran...,https://thumbs4.properati.com/8/tPtN9nsSZNL2zy...,NaN
97033,97033,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/1bjht_venta_depart...,MONOAMBIENTE CON BALCÓN EN PISO 12.ANTICIPO 35...,Departamento - Belgrano,NaN,0.0


In [733]:
data.property_type.value_counts()

apartment    71065
house        40268
PH            5751
store         4136
Name: property_type, dtype: int64

In [758]:
data_mask = data["rooms_completo"].isnull()
data_encontrada = data_room_null_description[data_mask].apply(lambda x: x if x is np.NaN else x.count("dormitorio"))
data_encontrada_mask = data_encontrada.notnull()
data_encontrada[data_encontrada_mask]
#data[data_encontrada_mask,"rooms_completo"]= 
#data["rooms_completo"].isnull().value_counts()
# encontradas_mask = encontradas.isnull()
# encontradas[encontradas_mask]
# encontradas

1         2
27        0
47        0
49        0
57        0
         ..
121157    0
121160    4
121209    4
121214    0
121219    0
Name: description, Length: 18732, dtype: int64

In [735]:
#display(data.loc[:,["property_type","rooms_completo","description"]])
#data.pivot_table('survived', index='sex', columns='class', margins=True, margins_name= "totales")
data.pivot_table(["rooms"],index='rooms_completo', columns="property_type", margins=True)

rooms                                            
property_type          PH  apartment      house      store        All
rooms_completo                                                       
0.0              1.670330   1.239905   4.208333   5.000000   1.352416
1.0              1.000000   1.000000   1.000000   1.000000   1.000000
2.0              2.000000   2.000000   2.000000   2.000000   2.000000
3.0              3.000000   3.000000   3.000000   3.000000   3.000000
4.0              4.000000   4.000000   4.000000   4.000000   4.000000
5.0              5.000000   5.000000   5.000000   5.000000   5.000000
6.0              6.000000   6.000000   6.000000   6.000000   6.000000
7.0              7.000000   7.000000   7.000000   7.000000   7.000000
8.0              8.000000   8.000000   8.000000   8.000000   8.000000
9.0              9.000000   9.000000   9.000000        NaN   9.000000
10.0            10.000000  10.000000  10.000000  10.000000  10.000000
11.0            11.000000  11.000000  11.000000        NaN  11.000000
12.0            12.000000  12.000000  12.000000        NaN  12.000000
13.0            13.000000  13.000000  13.000000        NaN  13.000000
14.0                  NaN  14.000000  14.000000        NaN  14.000000
15.0                  NaN  15.000000  15.000000        NaN  15.000000
16.0                  NaN        NaN  16.000000        NaN  16.000000
17.0                  NaN  17.000000  17.000000        NaN  17.000000
18.0                  NaN  18.000000  18.000000        NaN  18.000000
19.0                  NaN        NaN  19.000000        NaN  19.000000
20.0            20.000000  20.000000  20.000000        NaN  20.000000
21.0                  NaN  21.000000  21.000000        NaN  21.000000
22.0                  NaN  22.000000  22.000000        NaN  22.000000
23.0                  NaN        NaN  23.000000        NaN  23.000000
24.0                  NaN        NaN  24.000000        NaN  24.000000
25.0                  NaN  25.000000  25.000000        NaN  25.000000
27.0                  NaN        NaN  27.000000        NaN  27.000000
28.0                  NaN        NaN  28.000000        NaN  28.000000
29.0                  NaN        NaN  29.000000        NaN  29.000000
30.0                  NaN        NaN  30.000000        NaN  30.000000
31.0                  NaN        NaN  31.000000        NaN  31.000000
32.0                  NaN  32.000000  32.000000        NaN  32.000000
All              3.217431   2.508667   4.452388   2.438095   3.080840